# Interactive Labeling Tool with OpenAI Assistants
> Functionality for in-the-loop labeling 

In [ ]:
#| default_exp interactive_labeling

In [ ]:
import pandas as pd
from IPython.display import display, clear_output
import time
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def read_csv(file_path):
    import pandas as pd
    df = pd.read_csv(file_path)
    return df['Text'].tolist()

In [ ]:
#| export
def process_lines(lines, assistant_manager, context=""):
    data = []
    additional_context = "Return a list of labels for each utterance. Each utterance is separated by \n"
    context += additional_context
    assistant_manager.create_thread(context)

    all_lines = "\n ".join(lines)

    try:
        completed_run = assistant_manager.submit_message(all_lines)
        response_page = assistant_manager.get_response()
        messages = [msg for msg in response_page] 
        assistant_message = messages[-1].content[0].text.value
        labels = assistant_message.replace('\n', ' ').replace(',', ' ').split()

        # Check if labels are one of the specified labels
        valid_labels = ["NEU", "OTR", "PRS", "REP"]
        labels = [label if label in valid_labels else "NEU" for label in labels]

    except Exception as e:
        # Handle any exception that occurred during API call
        print(f"An error occurred: {e}")
        # Default to "NEU" for all lines in case of an error
        labels = ["NEU"] * len(lines)

    # append tuple (line, label) to data using zip
    data = list(zip(lines, labels))

    return(data)

In [ ]:
#| export
def label_data(unlabeled_text):
    """
    Prompts the user to label the given input.

    Args:
    unlabeled_text (str): The text data that needs labeling.

    Returns:
    str: The label provided by the user.
    """
    #print(f"Label the following line: {unlabeled_text}")
    label = input("Enter the correct label: ")
    while label not in ["NEU", "OTR", "PRS", "REP"]:
        print("Invalid label. Please enter a valid label.")
        label = input("Enter the correct label: ")

    return label

In [ ]:
#| export
def create_df_from_list(text_list):
    import pandas as pd 
    return pd.DataFrame(text_list, columns=["Text"])

In [ ]:
#| export
def increase_batch_size(batch_size, accuracy):
    if accuracy <= 0.90:
        batch_size = 5
    elif accuracy > 0.90:
        batch_size = 10
    return batch_size

In [ ]:
#| export
def process_batch(unlabeled_data, start_index, batch_size, batch_sizes):
    end_of_batch = min(start_index + batch_size, len(unlabeled_data))
    actual_batch = unlabeled_data[start_index:end_of_batch]
    actual_batch_size = len(actual_batch)
    batch_sizes.append(actual_batch_size)  # Actual number of items in the batch
    return actual_batch, actual_batch_size


In [ ]:
#| export
def get_user_labels(batch, assistant_manager, context):
    import time 
    predictions = process_lines(batch, assistant_manager, context)
    labeled_data = []
    correct_responses = 0

    for text, prediction in predictions:
        print(f"Predicted for '{text}': {prediction}")
        time.sleep(1)
        user_decision = input(f"Do you agree with this label? (Y/N): ")

        correct_label = label_data(text) if user_decision.lower() != 'y' else prediction
        correct_responses += correct_label == prediction
        context += f"\nuser: '{text}'\nassistant: '{correct_label}'"
        labeled_data.append((text, correct_label))

    return labeled_data, correct_responses



In [ ]:
#| export
def calculate_accuracy(correct_responses, batch_size):
    return correct_responses / batch_size if batch_size > 0 else 0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()